In [16]:
## Importing necessary libraries

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd
import math
from collections import Counter ##outputs frequency dictionary

In [17]:
## Tree class for decision tree

class TreeNode:
    def __init__(self, data,output):
        self.data = data 
        self.children = {} ##dictionary to store child nodes
        self.output = output
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

In [18]:
## Decision tree class for building decision tree

class Decision_tree:
    def __init__(self):
        self.__root = None
        

    def calc_entropy(self,rows): ## Function to calculate entropy
        entropy = 0
        freq = Counter(rows) ## frequency dictionary
        total_values = len(rows)
        for i in freq:
            proportion = freq[i]/total_values
            entropy += -proportion*math.log2(proportion)
        return entropy
        
        
    def gain_ratio(self,X,Y,selected_feature): ## Gain ratio calculations
        
    
        entropy_before_splitting = self.calc_entropy(Y)

        entropy_after_splitting = 0
        split_info = 0
        df = pd.DataFrame(X)   ##storing in dataframe for easy access of corresponding values of input and output
        df['target'] = Y  
        size = len(x)
        counter = Counter(X[:,selected_feature])

        for i in counter:
            df_req = df[df[selected_feature] == i]

            entropy_after_splitting += (counter[i]/size)*self.calc_entropy(df_req['target'])

            split_info += (-counter[i]/size)*math.log2(counter[i]/size)

        if split_info == 0 :
                return math.inf


        info_gain = entropy_before_splitting - entropy_after_splitting
        gain_ratio = info_gain / split_info
        return gain_ratio

    
    
    
## Building decision tree     
        
        
    def __decision_trees(self,x,y,features,level,classes):
        
       ## Base case for recursion
    
        if len(set(y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in y:
                    output = i
                    print("Count of",i,"=",len(y))
                else :
                    print("Count of",i,"=",0)
                    
                    
            print("Current Entropy is =  0.0")
            

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        
        if len(features) == 0:
            print("Level",level)
            freq_map = Counter(y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

        
            print("Current Entropy  is =",self.calc_entropy(y))
                        

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        
        ## main code
        
        
        max_gain = -math.inf
        final_feature = None
        for f in features :
            
            current_gain = self.gain_ratio(x,y,f)
            
            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freq_map = Counter(y)
        output = None
        max_count = -math.inf

        for i in classes:
            if i not in freq_map:
                print("Count of",i,"=",0)
            else :
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])
                
        print("Current Entropy is =",self.calc_entropy(y))
        print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
        print()
        
        
        
        
        unique_values = set(x[:,final_feature])
        df = pd.DataFrame(x)
        df['target'] = y

        current_node = TreeNode(final_feature,output)

        
        new_features = [i for i in features if i!= final_feature]
        for i in unique_values:
            df1 = df[df[final_feature] == i]  ## Extracting required information
            new_x = df1.iloc[:,0:df1.shape[1]-1].values
            new_y = df1['target'].values
            
            node = self.__decision_trees(new_x,new_y,new_features,level+1,classes) ## Recursive call
            current_node.add_child(i,node)

        

        return current_node
    
    def fit(self,x,y):  ## Fit function for classifier
        
        features = [i for i in range(len(x[0]))]
        classes = set(y)
        
        self.__root = self.__decision_trees(x,y,features,0,classes)
        
        
        
    def export_tree_pdf(self,filename=None):  ## Function to write tree to pdf using pydotplus and graphviz modules
        import pydotplus
        from collections import deque
        
        dot_data = '''digraph Tree {
node [shape=box] ;'''
        
        queue = deque()
        
        r = self.__root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.data,r.output) 
        
        while len(queue) != 0 :
            node = queue.popleft()
            for i in node.children:
                count+=1
                if(node.children[i].index==-1):
                    node.children[i].index = count
                
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].data,node.children[i].output) 
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                queue.append(node.children[i])
        
        dot_data = dot_data + "\n}"

        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)    

In [19]:
## Input of xor 

xor = np.array([[1,1],
       [0,1],
       [1,0],
       [0,0]])
## output of xor

output = np.array([1,1,1,0])

In [20]:
## Calling classifier and saving tree in pdf file for xor

clf_xor = Decision_tree()
clf_xor.fit(xor,output)
clf_xor.export_tree_pdf(filename="xortree.pdf")

Level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature  X[0] with gain ratio 4.803949376929977

Level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on feature  X[1] with gain ratio 10.375139177111546

Level 2
Count of 0 = 1
Count of 1 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 2
Current Entropy is =  0.0
Reached leaf Node



In [21]:
## importing iris dataset

iris=datasets.load_iris()
x=iris.data
y=iris.target


In [22]:
## Calling classifier and saving tree in pdf file for iris
clf_iris = Decision_tree()

clf_iris.fit(x,y)
clf_iris.export_tree_pdf(filename="iristree.pdf")

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.584962500721156
Splitting on feature  X[3] with gain ratio 0.3545578136119045

Level 1
Count of 0 = 29
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 7
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 7
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 1
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 1
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 7
Count of 2 = 1
Current Entropy is = 0.5435644431995964
Splitting on feature  X[2] with gain ratio 1.5146541731593821

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node
